Note: Must have uploaded images of cards

In [ ]:
# Allows user to guess what card matches the given prompt

In [ ]:
import PIL
from PIL import Image

import pandas as pd
import requests

# Reading in CSV of cards
url = 'https://docs.google.com/spreadsheets/d/1AWHOzyVWws-VSe5JsMb4gmJHHB5vCMy0OXbPH4pPjwA/edit?usp=sharing'

r = requests.get(f'https://docs.google.com/spreadsheet/ccc?key=' + url.split('/')[-2] + '&output=csv')
open('dataset.csv', 'wb').write(r.content)
df = pd.read_csv('dataset.csv')

In [ ]:
# Reading in CSV of GPT prompts
url = 'https://docs.google.com/spreadsheets/d/1cPBBYhOHMSFVZDeXte4dau2R5iFhHiyc_LuMoj7TG_c/edit?usp=sharing'

r = requests.get(f'https://docs.google.com/spreadsheet/ccc?key=' + url.split('/')[-2] + '&output=csv')
open('dataset.csv', 'wb').write(r.content)
gptPrompts = pd.read_csv('dataset.csv')
gptPrompts.set_index('Card', inplace=True)

In [ ]:
def combineImages(index, row):
    numPlayers = row["PLAYERS"]
    cards = ["C1_CARD", "C2_CARD", "C3_CARD", "C4_CARD"]
    if numPlayers > 4:
        cards.append("C5_CARD")
    if numPlayers > 5:
        cards.append("C6_CARD")

    card_names = [str(int(card)) + ".jpg" for card in row[cards]]

    card_images = [Image.open(card) for card in card_names]

    card_images = [image.resize((1000, 1500)) for image in card_images]

    size = card_images[0].size

    new_image = Image.new('RGB',(numPlayers * size[0], size[1]), (250,250,250))

    for i in range(numPlayers):
        new_image.paste(card_images[i],(0 + 1000 * i, 0))

    return new_image

In [ ]:
def getCardLoc(row):
    imageNum = 1

    if row["TARGET"] == row["C1_CARD"]:
        imageNum = 1
    elif row["TARGET"] == row["C2_CARD"]:
        imageNum = 2
    elif row["TARGET"] == row["C3_CARD"]:
        imageNum = 3
    else:
        imageNum = 4
    return imageNum

Generate images and guess

In [ ]:
guesses = pd.DataFrame(columns=['rowNum','target', 'prompt', 'location','guess', 'correct'])

In [ ]:
import random
import time
i = 0

while True:
    r = random.randint(0, 23309)
    row = df.iloc[r]
    if row["PLAYERS"] == 4:
        print("Please wait 10 seconds for the next prompt and image to appear")
        correct = getCardLoc(row)

        curImage = combineImages(r, row)
        curImage.show()

        prompt = gptPrompts.loc[row["TARGET"]]["Prompt"]
        print("Prompt:", prompt)
        time.sleep(10) # Input seems to get called before image is shown, so sleep to make sure image is shown
        guess = input('What card matches the prompt best? ')
        correct = getCardLoc(row)

        print("The correct answer was: ", correct)

        new_guess = { 'rowNum': r, 'target': row["TARGET"], 'prompt':prompt, 'location': correct, 'guess': guess, 'correct': 1 if int(guess) == correct else 0}
        guesses.loc[len(guesses)] = new_guess

        keepGuessing = input("Enter y to go again, n to stop: ")

        if keepGuessing == "y":
            continue
        else:
            break

guesses